In [2]:
# env의 파이썬 경로 확인
!conda run -n cnidium-ssp python -c "import sys; print(sys.executable)"

# cnidium-ssp 환경에서 ipykernel 재등록
!conda run -n cnidium-ssp python -m ipykernel install --user --name cnidium-ssp --display-name "Python (cnidium-ssp)"


/opt/conda/envs/cnidium-ssp/bin/python

Installed kernelspec cnidium-ssp in /root/.local/share/jupyter/kernels/cnidium-ssp



In [3]:
!cat ~/.local/share/jupyter/kernels/cnidium-ssp/kernel.json


{
 "argv": [
  "/opt/conda/envs/cnidium-ssp/bin/python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "Python (cnidium-ssp)",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}

In [1]:
import sys, platform, os
print("Kernel exe:", sys.executable)           # ← /opt/conda/envs/cnidium-ssp/bin/python
print("sys.prefix:", sys.prefix)               # ← /opt/conda/envs/cnidium-ssp
!which python                                  # ← /opt/conda/envs/cnidium-ssp/bin/python


Kernel exe: /opt/conda/envs/cnidium-ssp/bin/python
sys.prefix: /opt/conda/envs/cnidium-ssp
/opt/conda/bin/python


In [2]:
# 목적: 전용 conda 커널 사용 보장 (이미 설치되어 있어도 안전)
# 입출력: 없음(설치/등록 로그)
# 주의: 셸 명령은 conda-run으로 env를 명시해 실행
!conda run -n cnidium-ssp python -V || conda create -n cnidium-ssp python=3.10 -y
!conda install -n cnidium-ssp -y numpy pandas scikit-learn matplotlib ipykernel
!conda run -n cnidium-ssp python -m ipykernel install --user --name cnidium-ssp --display-name "Python (cnidium-ssp)"


import sys, platform
print("Kernel exe:", sys.executable) # /opt/conda/envs/cnidium-ssp/bin/python 이어야 OK
print("OS:", platform.platform())

Python 3.10.18

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.7.0



# All requested packages already installed.

Installed kernelspec cnidium-ssp in /root/.local/share/jupyter/kernels/cnidium-ssp

Kernel exe: /opt/conda/envs/cnidium-ssp/bin/python
OS: Linux-6.8.0-40-generic-x86_64-with-glibc2.35


In [4]:
# 목적: /workspace/DataM 를 ROOT로 고정 + 표준 폴더 생성
from pathlib import Path
ROOT = Path("/workspace/DataM").resolve()
RAW = ROOT/"data"/"raw"; PROC = ROOT/"data"/"processed"
RPT = ROOT/"reports"; FIG = RPT/"figures"
EXP = ROOT/"experiments"; LOGS = ROOT/"logs"
for p in [RAW, PROC, RPT, FIG, EXP, LOGS]:
    p.mkdir(parents=True, exist_ok=True)
print("ROOT=", ROOT)

ROOT= /workspace/DataM


In [5]:
# 목적: RAW에 CSV가 없으면 샘플 생성(파이프라인 점검용)
from pathlib import Path; import pandas as pd
RAW = Path("/workspace/DataM/data/raw")
RAW.mkdir(parents=True, exist_ok=True)
if not list(RAW.glob("*.csv")):
    df = pd.DataFrame({
        "month": [1,2,3,4,5]*27,
        "CO2ppm":[400,405,410,395,402]*27,
        "Temp":[20,21,19,22,20]*27,
        "Humid":[60,62,58,63,59]*27,
        "VPD":[1.1,1.0,1.2,1.05,1.08]*27,
        "Leaf_TPC":[0.8,0.82,0.78,0.85,0.81]*27
    })
    fp = RAW/"sample_0924.csv"; df.to_csv(fp, index=False, encoding="utf-8-sig"); print("Wrote:", fp)
print("RAW CSVs:", [p.name for p in RAW.glob('*.csv')])

RAW CSVs: ['3. 천궁 샘플데이터.csv']


In [7]:
# 목적: 프로젝트 루트 고정 + 폴더 보장
# 입출력: 폴더 생성(project/data/raw 등)
# 주의: 경로는 본인 PC에 맞게 수정 가능
from pathlib import Path
ROOT = Path(r"/workspace/DataM").resolve()
RAW = ROOT/"data"/"raw"; PROC = ROOT/"data"/"processed"
RPT = ROOT/"reports"; FIG = RPT/"figures"
EXP = ROOT/"experiments"; LOGS = ROOT/"logs"
for p in [RAW, PROC, RPT, FIG, EXP, LOGS]:
    p.mkdir(parents=True, exist_ok=True)
print("ROOT=", ROOT)

ROOT= /workspace/DataM


In [8]:
# 목적: MRE에 필요한 최소 패키지 설치 확인
# 입출력: 없음(설치 로그), 버전 프린트
# 주의: 노트북 커널이 conda env(cnidium-ssp)인지 확인
import sys, subprocess
pkgs = ["numpy", "pandas", "scikit-learn", "matplotlib"]
subprocess.run([sys.executable, "-m", "pip", "install", *pkgs, "-q"]) # 조용히 설치
import numpy as np, pandas as pd, sklearn, matplotlib
print("NumPy", np.__version__, "Pandas", pd.__version__, "Sklearn", sklearn.__version__, "Matplotlib", matplotlib.__version__)

NumPy 1.26.4 Pandas 2.3.2 Sklearn 1.7.2 Matplotlib 3.10.6


In [9]:
# 목적: joblib 임시폴더를 영문 경로로 고정(향후 병렬 실행 대비)
# 입출력: 없음
import os
TMP = ROOT/"tmp_joblib"; TMP.mkdir(exist_ok=True)
os.environ["JOBLIB_TEMP_FOLDER"] = str(TMP)

In [12]:
# 목적: Leaf_TPC 5-Fold CV 실행 → metrics.json 저장 (수정: 괄호 오타 fix)
import json
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.model_selection import KFold, cross_validate
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet

SEED = 42

# 경로 가드
try:
    RAW, EXP
except NameError:
    ROOT = Path("/workspace/DataM").resolve()
    RAW  = ROOT / "data" / "raw"
    EXP  = ROOT / "experiments"
    RAW.mkdir(parents=True, exist_ok=True)
    EXP.mkdir(parents=True, exist_ok=True)

# 데이터 로드
csvs = sorted(RAW.glob("*.csv"))
assert csvs, f"No CSV in {RAW}"
src = csvs[0]
try:
    df = pd.read_csv(src, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(src, encoding="cp949")
print(f"Loaded: {src.name}  rows={len(df)}  cols={len(df.columns)}")

# 특성
num_cand = [
    "CO2ppm","Temp","Humid","VPD","Chl_a","Chl_b","TChl","Car",
    "ABS-RC","Dio-RC","Tro-RC","Eto-RC","PI_abs","DF_abs","SFI_abs","Fv-Fm",
    "Leaf_ExtractionYield","Root_ExtractionYield"
]
num_cols = [c for c in num_cand if c in df.columns]
if "month" not in df.columns:
    df["month"] = 1

target = "Leaf_TPC" if "Leaf_TPC" in df.columns else df.columns[-1]
X = df[["month"] + num_cols].copy()
y = df[target].copy()

# 파이프라인
month_cycle = FunctionTransformer(
    lambda X_: np.c_[np.sin(2*np.pi*(X_.values[:, 0]/12.0)),
                     np.cos(2*np.pi*(X_.values[:, 0]/12.0))],
    feature_names_out="one-to-one"
)
trans = [("month_cycle", month_cycle, ["month"])]
if num_cols:
    trans.append(("scale", StandardScaler(), num_cols))

prep = ColumnTransformer(trans, remainder="drop")
pipe = Pipeline([("prep", prep), ("model", ElasticNet(random_state=SEED))])

# 5-Fold CV
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
scoring = {"MAE": "neg_mean_absolute_error", "MSE": "neg_mean_squared_error", "R2": "r2"}
res = cross_validate(pipe, X, y, cv=cv, scoring=scoring, return_train_score=False)

def mstd(a):
    a = np.asarray(a)
    return float(a.mean()), float(a.std())

mae_m, mae_s = mstd(-res["test_MAE"])
mse_m, mse_s = mstd(-res["test_MSE"])
r2_m,  r2_s  = mstd( res["test_R2"])

out = {
    "target": target,
    "n_samples": int(len(df)),
    "n_features": int(X.shape[1]),   # ← 여기 괄호 수정!
    "cv": "KFold(n_splits=5, shuffle=True, random_state=42)",
    "metrics": {
        "MAE_mean": mae_m, "MAE_std": mae_s,
        "MSE_mean": mse_m, "MSE_std": mse_s,
        "R2_mean":  r2_m,  "R2_std":  r2_s
    }
}

exp_dir = EXP / "exp_20250924_mre"
exp_dir.mkdir(parents=True, exist_ok=True)
save_path = exp_dir / "metrics.json"
save_path.write_text(json.dumps(out, ensure_ascii=False, indent=2), encoding="utf-8-sig")

print(out)
print("Saved ->", save_path)


Loaded: 3. 천궁 샘플데이터.csv  rows=135  cols=25
{'target': 'Leaf_TPC', 'n_samples': 135, 'n_features': 19, 'cv': 'KFold(n_splits=5, shuffle=True, random_state=42)', 'metrics': {'MAE_mean': 0.5573793397507052, 'MAE_std': 0.04497281463008568, 'MSE_mean': 0.38196725768321516, 'MSE_std': 0.058693079020913703, 'R2_mean': 0.20440509362505335, 'R2_std': 0.04596065559560122}}
Saved -> /workspace/DataM/experiments/exp_20250924_mre/metrics.json


In [13]:
# 목적: 오늘 환경(버전/OS/seed) 기록 → 재현성 확보
# 입출력: logs/setup_20250924.md
# 주의: utf-8-sig로 저장해 윈도우에서도 한글 깨짐 방지
import sys, platform, json
p = LOGS/"setup_20250924.md"
meta = {"python": sys.version, "platform": platform.platform(), "seed": 42}
p.write_text("# 환경 기록 (2025-09-24)\n\n```\n"+json.dumps(meta, ensure_ascii=False, indent=2)+"\n```\n", encoding="utf-8-sig")
p

PosixPath('/workspace/DataM/logs/setup_20250924.md')

In [ ]:
%cd /workspace/DataM

# .gitignore 생성/갱신
ignore = """__pycache__/
*.pyc
.ipynb_checkpoints/
data/processed/
reports/figures/
tmp_joblib/
.DS_Store
"""
open(".gitignore","w",encoding="utf-8").write(ignore)

# 이미 추적 중인 체크포인트를 인덱스에서 제거(파일은 로컬에 남음)
!git rm -r --cached .ipynb_checkpoints 2>/dev/null || true

# 변경분 스테이징/커밋
!git add -A
!git commit -m "chore: update notebook & env log; add .gitignore (checkpoints/artifacts)"
